In [ ]:
# ! pip install -e /home/jovyan/downscaling/xclim/

In [1]:
%matplotlib inline
import xarray as xr
import scipy as sp
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import os 
import gcsfs

import dask.distributed as dd
import dask
import rhg_compute_tools.kubernetes as rhgk

In [2]:
from xclim import sdba

In [3]:
from xclim.sdba.adjustment import AnalogQuantilePreservingDownscaling, QuantileDeltaMapping
from xclim.sdba.utils import equally_spaced_nodes
from xclim.core.calendar import convert_calendar
from xclim import set_options

In [4]:
def convert_to_noleap_calendar(ds, target='noleap'):
    ds_noleap = convert_calendar(ds, target=target)
    return ds_noleap

In [5]:
'''tas_eager = xr.open_dataset('/home/jovyan/analog_method_prototype_wastate_era5_data.nc')
tas_eager = convert_calendar(tas_eager, target='noleap')
ref = tas_eager['tas']
hist = tas_eager['tas'] + 3
sim = tas_eager['tas'] + 5'''

"tas_eager = xr.open_dataset('/home/jovyan/analog_method_prototype_wastate_era5_data.nc')\ntas_eager = convert_calendar(tas_eager, target='noleap')\nref = tas_eager['tas']\nhist = tas_eager['tas'] + 3\nsim = tas_eager['tas'] + 5"

In [16]:
np.random.seed(0)

lon = [-99.83, -99.32, -99.79, -99.23]
lat = [42.25, 42.21, 42.63, 42.59]
# time = pd.date_range(start='1994-12-17', end='2015-01-15')
# time = pd.date_range(start='1995-01-01', end='2014-12-31')
time = xr.cftime_range(start="1994-12-17", end="2015-01-15", calendar="noleap")
temperature_ref = 15 + 8 * np.random.randn(len(time), 4, 4)
temperature_train = 15 + 8 * np.random.randn(len(time), 4, 4)
variable = "scen"

ref_fine = xr.Dataset(
     data_vars=dict(
         scen=(["time", "lat", "lon"], temperature_ref),
     ),
     coords=dict(
         time=time,
         lon=(["lon"], lon),
         lat=(["lat"], lat),
         # reference_time=reference_time,
     ),
     attrs=dict(description="Weather related data."),
     )

ds_train = xr.Dataset(
     data_vars=dict(
         temperature=(["time", "lat", "lon"], temperature_train),
     ),
     coords=dict(
         time=time,
         lon=(["lon"], lon),
         lat=(["lat"], lat),
         # reference_time=reference_time,
     ),
     attrs=dict(description="Weather related data."),
     )

In [17]:
ref_fine

<xarray.Dataset>
Dimensions:  (time: 7330, lat: 4, lon: 4)
Coordinates:
  * time     (time) object 1994-12-17 00:00:00 ... 2015-01-15 00:00:00
  * lon      (lon) float64 -99.83 -99.32 -99.79 -99.23
  * lat      (lat) float64 42.25 42.21 42.63 42.59
Data variables:
    scen     (time, lat, lon) float64 29.11 18.2 22.83 ... 12.84 2.884 14.02
Attributes:
    description:  Weather related data.

In [18]:
# ds = xr.tutorial.load_dataset("air_temperature").resample(time="D").mean()
# temp_slice = convert_to_noleap_calendar(ds['air'][:, :2, :2], target='noleap')
'''ref_fine = convert_to_noleap_calendar(ref_fine['temperature'], target='noleap')
ds_train = convert_to_noleap_calendar(ds_train['temperature'], target='noleap')'''

"ref_fine = convert_to_noleap_calendar(ref_fine['temperature'], target='noleap')\nds_train = convert_to_noleap_calendar(ds_train['temperature'], target='noleap')"

In [19]:
# temp_slice_mean = temp_slice.mean(['lat', 'lon'])
ds_ref_coarse = ref_fine.mean(['lat', 'lon'])
ds_train = ds_train.mean(['lat', 'lon'])
# temp_slice_mean_resampled = temp_slice_mean.broadcast_like(temp_slice)
ref_coarse = ds_ref_coarse.broadcast_like(ref_fine)
# temp_slice_mean_climatechange = temp_slice_mean_resampled + 3
ds_bc = ds_train + 3

In [20]:
ref_coarse

<xarray.Dataset>
Dimensions:  (time: 7330, lat: 4, lon: 4)
Coordinates:
  * time     (time) object 1994-12-17 00:00:00 ... 2015-01-15 00:00:00
  * lat      (lat) float64 42.25 42.21 42.63 42.59
  * lon      (lon) float64 -99.83 -99.32 -99.79 -99.23
Data variables:
    scen     (time, lat, lon) float64 20.32 20.32 20.32 ... 11.15 11.15 11.15

In [ ]:
with set_options(sdba_extra_output=True):
    quantiles = equally_spaced_nodes(620, eps=None)
    QDM = QuantileDeltaMapping(kind='+', nquantiles=quantiles, 
                               group=sdba.Grouper("time.dayofyear", window=31))
    # QDM.train(temp_slice_mean + 2, temp_slice_mean)
    QDM.train(ds_ref_coarse, ds_train)
    # fake_biascorrected = QDM.adjust(temp_slice_mean + 4)
    fake_biascorrected = QDM.adjust(ds_bc)
    fake_biascorrected = fake_biascorrected['scen'].assign_coords(sim_q=fake_biascorrected.sim_q).to_dataset()

In [ ]:
'''print(ds_ref_coarse)
print(ds_train)
print(ds_bc)'''
print(fake_biascorrected)

In [ ]:
# biascorrected = fake_biascorrected['scen'].broadcast_like(temp_slice)

In [ ]:
biascorrected[:, 1, 0]

In [ ]:
year = 2005
biascorrected_year = biascorrected.sel(time=slice("{}-01-01".format(year), "{}-12-31".format(year)))

In [ ]:
'''AIQPD = AnalogQuantilePreservingDownscaling(kind='+', nquantiles=62, 
                                            group=sdba.Grouper("time.dayofyear", window=31))'''
AIQPD = AnalogQuantilePreservingDownscaling(kind='+', nquantiles=620, 
                                            group=sdba.Grouper("time.dayofyear", window=31))
'''AIQPD = AnalogQuantilePreservingDownscaling(kind='+', nquantiles=620, 
                                            group=sdba.Grouper("time.dayofyear"))'''

In [ ]:
'''print(ds['temperature'].shape)
print(temp_slice_mean_resampled.shape)

print(20*365)
print(31*20)

print(temp_slice.shape)
print(temp_slice_mean_resampled.shape)'''

In [ ]:
# AIQPD.train(ref[:5000, :, :], hist[:5000, :, :])
# AIQPD.train(temp_slice_mean_resampled, temp_slice)
AIQPD.train(ref_coarse, ref_fine)
AIQPD

In [ ]:
AIQPD.ds

In [ ]:
print(AIQPD.ds.af.shape)

In [ ]:
AIQPD.ds.af

In [ ]:
'''len(np.unique(biascorrected.time.dt.year))
365*20'''

In [ ]:
# AIQPD.ds.af[0, 0, 0, :].isel(dayofyear=352)
# AIQPD.ds.af[0, 0, :, :].isel(dayofyear=362)

In [ ]:
biascorrected_year.chunks

In [ ]:
AIQPD.ds.af.shape

In [ ]:
AIQPD.ds.af

In [ ]:
# aiqpd_downscaled = AIQPD.adjust(biascorrected)
aiqpd_downscaled = AIQPD.adjust(biascorrected_year)

In [ ]:
sim_year_slice = ds_bc.sel(time=slice("{}-01-01".format(year), "{}-12-31".format(year)))

# pick a day
# print("bias corrected value is {}".format(temp_slice_mean_climatechange.isel(time=100).values[0][0]))
print("simulation value is {}".format(sim_year_slice.isel(time=100).values))
quantile = 0.75

print("test quantile is {}".format(quantile))

closest_quantile_index = np.argmin(np.abs(AIQPD.ds.af.quantiles.values - quantile))

'''downscaled = (AIQPD.ds.af.isel(dayofyear=100).isel(quantiles=closest_quantile_index) 
              + temp_slice_mean_climatechange.isel(time=100))'''

'''downscaled = (AIQPD.ds.af.isel(dayofyear=100).isel(quantiles=closest_quantile_index) 
              + ds_bc.isel(time=100))'''
downscaled = (AIQPD.ds.af.isel(dayofyear=100).isel(quantiles=closest_quantile_index) 
              + biascorrected_year.isel(time=100))

print("klugey downscaled mean value (where I apply adjustment factors to the bias corrected values from above) is {}".format(downscaled.mean().values))
# print("fake bias corrected value is {}".format(biascorrected.isel(time=100).values[0][0]))

print("fake bias corrected value is {}".format(biascorrected_year.isel(time=100).values[0][0]))

print("aiqpd adjusted downscaled values are {}".format(aiqpd_downscaled.isel(time=100)))

print("aiqpd adjusted downscaled value is {}".format(aiqpd_downscaled.isel(time=100).mean().values))